# MATSim Warm-Starting

PAM can load experienced MATSim plans, make modifications, then write new input MATSim plans. This allows for changes to be made to a population or network without invalidating the MATSim plans.

For example, consider the case where we have the outputs from a MATSim baseline simulation. We want to test a new scenario that makes a minor change to some parts of the network, this includes removing some links. We would like to reuse the output baseline plans. They contain our best initial guess for agent choices for times, modes and routes, including for unselected plans. However, we cannot resue these plans as they are, because they include routes that are now impossible due to our new scenario removing some links.

PAM allows us to read in these experienced plans, remove routes from agents that use these links, then write back to a valid format for our new scenario.

In [1]:
from pathlib import Path

from pam.read import read_matsim
from pam.write import write_matsim

## Load

In [2]:
population = read_matsim(
    plans_path=Path("data/example_data/output_experienced_plans.xml"), keep_non_selected=True
)
population.stats

{'num_households': 6, 'num_people': 6, 'num_activities': 23, 'num_legs': 18}

## Define Filters

In [3]:
removed_links = ["1-5", "5-1"]


def leg_filter(leg):
    for link in removed_links:
        if link in leg.route.network_route:
            return True
        if leg.route.get("start_link") in removed_links:
            return True
        if leg.route.get("end_link") in removed_links:
            return True


def plan_filter(plan):
    for leg in plan.legs:
        if leg_filter(leg):
            return True

    for act in plan.activities:
        if act.location.link in removed_links:
            return True

## Modify

In [4]:
for _, _, person in population.people():
    if plan_filter(person.plan):
        for leg in person.legs:
            leg.route.xml = {}
        for activity in person.activities:
            activity.location.link = None

    for plan in person.plans_non_selected:
        if plan_filter(plan):
            for leg in plan.legs:
                leg.route.xml = {}
            for activity in plan.activities:
                activity.location.link = None

## Write

In [5]:
write_matsim(population, Path("tmp/warm_plans.xml"))

If we check these "warm" plans we will find that the agent "Nick" has had their plan modified. But all other agents have maintained all the original information.

It is easy to get warm starting wrong. But it also has great potential to reduce MATSim compute in many cases. Common pitfalls include:

- failing to remove all the required information (remember that link and transit facility ids are also included in agent leg attributes and activity attributes)
- accidentally removing plans or activities (make sure that crop=False when reading plans)
- creating impossible plans (such as by using an unknown mode)

We have added three common methods for warm starting to the PAM cli:

- [crop](../../api/cli/#pam-crop) (but note this does not currently support unselected plans)
- [wipe_links](../../api/cli/#wipe-links)
- [wipe_all_links](../../api/cli/#wipe-all-links)